# LLM Model

In [36]:
#LLM Model
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

prompt = PromptTemplate.from_template("what is the famous tour place in {city}?")
chain = LLMChain(llm=model, prompt=prompt)
city = "Seoul"
chain.run(city)
                                           

'One of the most famous tourist attractions in Seoul is Gyeongbokgung Palace. It is a grand palace and the largest of the Five Grand Palaces built by the Joseon Dynasty. Another popular destination is N Seoul Tower, which offers panoramic views of the city. Other notable places to visit include Bukchon Hanok Village, Myeongdong shopping district, Dongdaemun Design Plaza, and the vibrant neighborhood of Hongdae.'

# Sequential Chain

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template("what is the famous tour place in {city}? Tell me the name of the place only without additional comments.")
prompt2 = PromptTemplate.from_template("How can I get {place} by {transport}?")
chain1 = LLMChain(llm=model,prompt=prompt1,output_key="place",verbose=True)
chain2 = LLMChain(llm=model,prompt=prompt2,verbose=True)

chain = SequentialChain(chains=[chain1,chain2]
                        ,input_variables=["city","transport"],verbose=True)

chain.run({'city':'Seoul','transport':'subway'})
                                           

/Users/terrycho/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
what is the famous tour place in Seoul? Tell me the name of the place only without additional comments.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
How can I get  

Gyeongbokgung Palace by subway?

> Finished chain.

> Finished chain.


'\n\nTo get to Gyeongbokgung Palace by subway, take Line 3 (orange line) or Line 5 (purple line) to Gyeongbokgung Station. From there, take Exit 5 and walk for about 5 minutes to reach the palace.'

# Sequential Advacned

In [39]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template("what is the famous tour place in {city}? Tell me the name of the place only without additional comments.")
prompt2 = PromptTemplate.from_template("What is the top 5 restaurant in the {place} in city {city} without additional comments?") #output : restaurants
prompt3 = PromptTemplate.from_template("What is the best one restaurant and food for family dinner among {restaurants} ?") #output : restaurant_information
prompt4 = PromptTemplate.from_template("How can I get the {place} by using {transport}?") #output : transport_information
final_prompt = PromptTemplate.from_template("""
Please summarize the tour information with reastaurant information and transportation by using the this information.
Restaurant informations : {restaurant_information}
Transport information : {transport_information}
""")
            
chain1 = LLMChain(llm=model,prompt=prompt1,output_key="place",verbose=True)    
chain2 = LLMChain(llm=model,prompt=prompt2,output_key="restaurants",verbose=True)
chain3 = LLMChain(llm=model,prompt=prompt3,output_key="restaurant_information",verbose=True)
chain4 = LLMChain(llm=model,prompt=prompt4,output_key="transport_information",verbose=True)
final_chain = LLMChain(llm=model,prompt=final_prompt,output_key="tour_summary",verbose=True)
chain = SequentialChain(chains=[chain1,chain2,chain3,chain4,final_chain]
                        ,input_variables=["city","transport"],verbose=True)
chain.run({'city':'Seoul','transport':'subway'})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
what is the famous tour place in Seoul? Tell me the name of the place only without additional comments.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is the top 5 restaurant in the  

Gyeongbokgung Palace in city Seoul without additional comments?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is the best one restaurant and food for family dinner among 

1. HwaDongJang (화동장)
2. Tosokchon Samgyetang (토속촌 삼계탕)
3. Gyeongbokgung Palace Korean Restaurant (경복궁 한정식당)
4. Jaha Son Mandu (자하손만두)
5. Gaehwaok (개화옥) ?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
How can I get the  

Gyeongbokgung Palace by using subway?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Please summarize the tour information with reastaurant information and transportation b

' \nThe recommended restaurant for a family dinner is Gyeongbokgung Palace Korean Restaurant, which offers traditional Korean cuisine and is located near the famous Gyeongbokgung Palace. For transportation, you can take the Seoul Subway Line 3 or Line 5 to either Gyeongbokgung Station (Exit 5) or Gwanghwamun Station (Exit 2) and walk to the palace. Navigation apps are also available to help you find the best route.'

# Router Chain

In [8]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains import LLMChain
from langchain.llms import OpenAI


OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

prompt_infos = [
    {
        "name":"restaurants",
        "description":"Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template
    },
    {
        "name":"transport",
        "description":"Good for guiding the transport to get the place",
        "prompt_template": transport_template
    },
    {
        "name":"destination",
        "description":"Good for recommending place to tour",
        "prompt_template": destination_template
    }
]

destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = LLMChain(llm = model, prompt=prompt,verbose=True)
    destination_chains[name] = chain
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
#print(router_template)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(model, router_prompt)
default_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

chain.run("What is best restaurant in Seoul?")



> Entering new MultiPromptChain chain...


/Users/terrycho/anaconda3/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


restaurants: {'input': 'What is best restaurant in Seoul?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
What is best restaurant in Seoul?


> Finished chain.

> Finished chain.


'\nThere are many great restaurants in Seoul, so it\'s hard to choose just one as the "best." However, some popular and highly recommended options include:\n\n1. Jungsik - a Michelin-starred restaurant that offers modern Korean cuisine with a twist\n2. Myeongdong Kyoja - a famous spot for traditional handmade noodles and dumplings\n3. Gwangjang Market - a must-visit for street food lovers, with a wide variety of Korean snacks and dishes\n4. Hanilkwan - known for its delicious hanjeongsik, a traditional Korean meal with multiple courses\n5. Maple Tree House - a popular restaurant for Korean barbecue, with high-quality meat and a cozy atmosphere\n6. Tosokchon - famous for its samgyetang (ginseng chicken soup) and other traditional dishes\n7. Garak Market - a seafood lover\'s paradise, with a huge selection of fresh seafood to choose from and have cooked on the spot\n8. Mingles - another Michelin-starred restaurant that combines Korean and international flavors in a creative way\n9. Gwang